Hackathon Korea 2022 Metal Project B 4Q design - ydkim

In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
import numpy as np

In [2]:
from qiskit_metal.qlibrary.terminations.launchpad_wb_driven import LaunchpadWirebondDriven
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from collections import OrderedDict
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.user_components.Round_Transmon import RoundTransmon
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
design = designs.DesignPlanar({}, True)
design.chips.main.size.size_x = '8mm'
design.chips.main.size.size_y = '8mm'
design.chips.main.size.size_z = '-0.5mm'
design.chips.main.material = 'Silicon'
gui = MetalGUI(design)

In [4]:
opt_Q = dict(pad_width = '500um',
             pad_height = '80um',
             pad_gap = '20um',
             pocket_width = '650um',
             pocket_height = '450um',
             inductor_width = '2um',
             qubit_fillet = '30um',
             pocket_fillet = '50um',
             connection_pads = dict(bus_R=dict(loc_W=+1, loc_H=+1, pocket_rise='0um', pad_gap='15um', pad_width='85um', pad_height='30um', pad_cpw_shift='10um'),
                                    bus_L=dict(loc_W=-1, loc_H=+1, pocket_rise='0um', pad_gap='15um', pad_width='77um', pad_height='30um', pad_cpw_shift='10um'),
                                    RO=dict(loc_W=0, loc_H=-1, pocket_rise='0um', pad_gap='40um', pad_width='97um', pad_height='20um', pad_cpw_shift='0um', cpw_extend='0um')))


Q01 = RoundTransmon(design, 'Q01', options=dict(pos_x=0, pos_y=0, orientation='0',**opt_Q))
gui.rebuild()
gui.autoscale()


In [5]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")
c1.sim.setup.max_passes = 30
c1.sim.min_converged_passes = 1
c1.sim.setup.percent_error = 0.2
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 30,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.2,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [6]:
c1.sim.run(components=['Q01'], open_terminations=[('Q01', 'RO'), ('Q01', 'bus_L'), ('Q01', 'bus_R')])
c1.sim.capacitance_matrix

INFO 02:40PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:40PM [load_ansys_project]: 	Opened Ansys App
INFO 02:40PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 02:40PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/ddute/Documents/Ansoft/
	Project:   hackathon
INFO 02:40PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 02:40PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:40PM [connect]: 	Connected to project "hackathon" and design "Design_q3d" 😀 

INFO 02:40PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 02:40PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:40PM [analyze]: Analyzing setup Setup
INFO 02:41PM [get_matrix]: Exporting matrix data to (C:\Users\ddute\AppData\Local\Temp\tmpm1m0b0hc.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 

,bus_L_connector_pad_Q01,bus_R_connector_pad_Q01,ground_main_plane,pad_bot_Q01,pad_top_Q01,RO_connector_pad_Q01
bus_L_connector_pad_Q01,39.08235,-0.05390,-29.36684,-1.07007,-8.08612,-0.07222
bus_R_connector_pad_Q01,-0.05390,40.25582,-29.81937,-1.13022,-8.72311,-0.07659
ground_main_plane,-29.36684,-29.81937,231.66971,-47.18005,-43.25720,-28.00204
pad_bot_Q01,-1.07007,-1.13022,-47.18005,100.39799,-41.45330,-7.26676
pad_top_Q01,-8.08612,-8.72311,-43.25720,-41.45330,104.81907,-1.09752
RO_connector_pad_Q01,-0.07222,-0.07659,-28.00204,-7.26676,-1.09752,36.90864


In [7]:
c1.setup.junctions = Dict({'Lj': 10, 'Cj': 2})
c1.setup.freq_readout = 7.4
c1.setup.freq_bus = [9,8]

c1.run_lom()
c1.lumped_oscillator_all

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.616592 [GHz]
EC 265.845348 [MHz]
EJ 16.339560 [GHz]
alpha -300.277995 [MHz]
dispersion 1.566833 [KHz]
Lq 9.995968 [nH]
Cq 72.862769 [fF]
T1 333.908025 [us]

**Coupling Properties**

tCqbus1 3.244962 [fF]
gbus1_in_MHz 49.524877 [MHz]
χ_bus1 -0.405284 [MHz]
1/T1bus1 424.999455 [Hz]
T1bus1 374.482699 [us]

tCqbus2 -3.332502 [fF]
gbus2_in_MHz -61.801336 [MHz]
χ_bus2 -0.195002 [MHz]
1/T1bus2 17.735484 [Hz]
T1bus2 8973.814348 [us]

tCqbus3 -3.607558 [fF]
gbus3_in_MHz -59.479975 [MHz]
χ_bus3 -0.343891 [MHz]
1/T1bus3 33.908122 [Hz]
T1bus3 4693.711524 [us]
Bus-Bus Couplings
gbus1_2 2.980946 [MHz]
gbus1_3 2.849501 [MHz]
gbus2_3 3.646673 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.956828,301.065817,16.33956,-343.389104,5.724291,"[52.29160687554066, -65.1978785966081, -62.275...","[-0.7391615718410759, -0.29662483173236925, -0...",0.739162,52.291607
2,5.917711,296.890926,16.33956,-338.241235,4.970144,"[51.09551157834478, -60.97099805845892, -56.79...","[-0.664689289482409, -0.2500889974853835, -0.4...",0.664689,51.095512
3,5.840727,288.77047,16.33956,-328.257302,3.742728,"[50.06626844148013, -61.774597622704015, -58.9...","[-0.5687629023191515, -0.23901592533627455, -0...",0.568763,50.066268
4,5.763623,280.764027,16.33956,-318.451052,2.795536,"[49.73886911035268, -61.87963184994138, -58.06...","[-0.5018937552009786, -0.223413239724237, -0.3...",0.501894,49.738869
5,5.724555,276.755423,16.33956,-313.555315,2.40399,"[49.42409562240734, -61.293064033587825, -58.8...","[-0.4687918556670966, -0.21151443919798, -0.38...",0.468792,49.424096
6,5.685101,272.739941,16.33956,-308.66053,2.059797,"[49.315584107275406, -61.41436277046287, -58.8...","[-0.44161858424488143, -0.2048694997947392, -0...",0.441619,49.315584
7,5.663506,270.556009,16.33956,-306.002291,1.890998,"[49.30416105768423, -61.37215686421426, -59.03...","[-0.4283836560593565, -0.2006232411797738, -0....",0.428384,49.304161
8,5.644377,268.629675,16.33956,-303.659895,1.752095,"[49.30959360803073, -61.44252637415169, -58.98...","[-0.4173296537205049, -0.19763538143255446, -0...",0.417330,49.309594
9,5.634932,267.681318,16.33956,-302.507497,1.68699,"[49.33400038155429, -61.476400417508074, -59.0...","[-0.4123651809847036, -0.1961732480688835, -0....",0.412365,49.334000
10,5.625736,266.759888,16.33956,-301.388317,1.625734,"[49.39635905288555, -61.592462915920926, -59.2...","[-0.40824206644044786, -0.19528822880841806, -...",0.408242,49.396359


In [8]:
c1.sim.close()